<center>
<img style="float: center;" src="images/CI_horizontal.png" width="400">
</center>
<center>
    <span style="font-size: 1.5em;">
        <a href='https://www.coleridgeinitiative.org'>Website</a>
    </span>
</center>

<a href="https://doi.org/10.5281/zenodo.6324207"><img src="https://zenodo.org/badge/DOI/10.5281/zenodo.6324207.svg" alt="DOI"></a>


<center> Josh Edelmann, Benjamin Feder, Nathan Barrett </center>

# **Machine Learning Part 1: Data Preparation**

Whereas unsupervised machine learning can be used to understand underlying groupings within the data, supervised machine learning focuses on prediction of a certain outcome. With prediction, we will formulate our model based on training data, and then evaluate the results of the model on a separate test set. Similar to unsupervised machine learning, though, before running any algorithm, we must think critically about the features we want to include in our model. The variables we consider as features do not have to be limited to the variables in the provided tables, as there may be features that can be developed by leveraging variables across multiple sources. Additionally, we will want to make sure our input information is in a specific format so that it can be seamlessly used in modeling. This notebook will prepare the training and testing tables and the next notebook will cover prediction algorithms.

To this point, we have already tracked a cohort of bachelor's degree earners in Texas and explored their employment outcomes. We created various outcome measures and compared them amongst subgroups such as major and gender. Now, we will focus on one outcome measure, the presence of wages in the fourth quarter after graduation, and create a handful of features as our predictors. 

## **Learning Objectives**

We will create two separate tables to predict future employment in the fourth quarter after graduation. Our training set will consist of 2017 calendar year bachelor's degree recipients in Texas, and our testing set will include 2018 recipients. Eventually, both sets will contain the same features as well as the future outcome measure. One row in these tables will correspond to one graduate.

After you finish this notebook, you should know:
- How to create the label (outcome variable, Y)
- How to create features (predictor variables, X)

## **Import Packages and Set Up**

In [ ]:
# Database interaction imports
library(odbc)

# For data manipulation/visualization
library(tidyverse)

# For faster date conversions
library(lubridate)

In [ ]:
# Connect to the server
con <- DBI::dbConnect(odbc::odbc(),
                     Driver = "SQL Server",
                     Server = "msssql01.c7bdq4o2yhxo.us-gov-west-1.rds.amazonaws.com",
                     Trusted_Connection = "True")

## **Feature Creation**

We hope to create and/or identify the following features to include in our supervised machine learning example for each individual in both the training and testing sets:

- `gradtypi`, the type of institution from which the student graduated
- `gradmaj`, the 2-digit CIP code of the student
- `instregion`, the region of the institution in Texas
- `transfer_ind`, if the student was a transfer student sometime in the 2 years before graduation
- `total_sems`, the total number of semesters enrolled in the 2 years before graduation
- `total_hours`, the total number of credit hours enrolled in the 2 years before graduation
- `employed_count`, the number of individuals in the most recently measured cohort that were employed in their fourth quarter after graduation by `gradmaj` and `instregion`
- `employed_prop`, the proportion of individuals in the most recently measured cohort that were employed in their fourth quarter after graduation by `gradmaj` within `instregion`.

Notice how these features can be split into two groups:
1. Individual-based characteristics
2. Information from the past

> We will demonstrate how to construct these features for the training set of 2017 calendar year bachelor's degree recipients in Texas. We have already saved a permanent table, `grads17_dated`, with these individuals by slightly modifying the code used to create `grads15_dated` in the Data Exploration: Wages notebook.

### **Individual-Based Characteristics**

The individual-based characteristics can be identified directly from the individual, and do not rely on either past or future information to fill. Almost all of our desired features are individual-based, as all but `employed_count` and `employed_prop` can satisfy this constraint.

We will be able to retrieve this information by starting with the base table of 2017 calendar year bachelor's degree recipients in Texas.

In [ ]:
# see grads17_dated
qry <- "
SELECT TOP 5 *
FROM tr_tx_2021.dbo.grads17_dated
"
dbGetQuery(con, qry)

Given the desired features, though, we will need to join `grads17_dated` to both the `enrollments` and `institutions` tables to provide more detail regarding the graduating institution and prior enrollment history for this cohort. To limit the enrollment information to just the previous two years before graduation, we will first create a variable that will eventually bound the enrollment information.

In [ ]:
# create previous_2_years cutoff for each individual
qry <- "
SELECT TOP 5 *, dateadd(yyyy ,-2, grad_date) as 'previous_2_years'
FROM tr_tx_2021.dbo.grads17_dated cohort
"
dbGetQuery(con, qry)

We can then build out the query in a common table expression (CTE) to eventually take the final form.

> Click [here](#final-query) to skip directly to the complete query.

In [ ]:
# limit to variables that will help calculate features 
# left join to enrollments and institutions to preserve all members of the cohort
qry <- "
SELECT TOP 5 gradid, gradfice, grad_date, substring(gradmaj, 1, 2) as gradmaj, gradtypi, instregion, stufttr, stutotsch, stusem, stuyear, dateadd(yyyy ,-2, grad_date) as 'previous_2_years'
FROM tr_tx_2021.dbo.grads17_dated cohort
LEFT JOIN ds_tx_thecb.dbo.institutions inst
ON cohort.gradfice = inst.instfice 
LEFT JOIN ds_tx_thecb.dbo.enrollments enroll
ON cohort.gradid = enroll.studentid 
"
dbGetQuery(con, qry)

In [ ]:
# add in new variable mapping quarters of enrollment to semesters
# this will allow us to provide the 2 year cutoff
qry <- "
SELECT TOP 5 gradid, gradfice, grad_date, substring(gradmaj, 1, 2) as gradmaj, gradtypi, instregion, stufttr, stutotsch, stusem, stuyear, dateadd(yyyy ,-2, grad_date) as 'previous_2_years',
CASE 
    WHEN stusem = 2 THEN 4 
    WHEN stusem = 1 THEN 10 
    when stusem = 3 THEN 7 
    ELSE 7 
    END as new_month 
FROM tr_tx_2021.dbo.grads17_dated cohort
LEFT JOIN ds_tx_thecb.dbo.institutions inst
ON cohort.gradfice = inst.instfice 
LEFT JOIN ds_tx_thecb.dbo.enrollments enroll
ON cohort.gradid = enroll.studentid
"
dbGetQuery(con, qry)

In [ ]:
# find dates of enrollments
# note: in the enrollments file, sometimes the year is null, so the ordinary date conversion won't work
qry <- "
with new_table as (
    SELECT gradid, gradfice, grad_date, SUBSTRING(gradmaj, 1, 2) as gradmaj, gradtypi, instregion, stufttr, stutotsch, stusem, stuyear, dateadd(yyyy ,-2, grad_date) as 'previous_2_years',
    CASE 
        WHEN stusem = 2 THEN 4 
        WHEN stusem = 1 THEN 10 
        when stusem = 3 THEN 7 
        ELSE 7 
        END as new_month 
    FROM tr_tx_2021.dbo.grads17_dated cohort
    LEFT JOIN ds_tx_thecb.dbo.institutions inst
    ON cohort.gradfice = inst.instfice 
    LEFT JOIN ds_tx_thecb.dbo.enrollments enroll
    ON cohort.gradid = enroll.studentid 
)
SELECT top 5 gradid, gradfice, gradtypi, grad_date, gradmaj, instregion, stufttr, previous_2_years, stutotsch, case 
WHEN stuyear is null THEN 0 
ELSE CONVERT(datetime, CONCAT((new_month), '/', '01', '/', stuyear)) end as enroll_date
FROM new_table
"
dbGetQuery(con, qry)

In [ ]:
# filter for all enrollments in those two years prior to graduation
qry <- "
with new_table as (
    SELECT gradid, gradfice, grad_date, SUBSTRING(gradmaj, 1, 2) as gradmaj, gradtypi, instregion, stufttr, stutotsch, stusem, stuyear, dateadd(yyyy ,-2, grad_date) as 'previous_2_years',
    CASE 
        WHEN stusem = 2 THEN 4 
        WHEN stusem = 1 THEN 10 
        when stusem = 3 THEN 7 
        ELSE 7 
        END as new_month 
    FROM tr_tx_2021.dbo.grads17_dated cohort
    LEFT JOIN ds_tx_thecb.dbo.institutions inst
    ON cohort.gradfice = inst.instfice 
    LEFT JOIN ds_tx_thecb.dbo.enrollments enroll
    ON cohort.gradid = enroll.studentid 
),
combined as (
    SELECT gradid, gradfice, gradtypi, grad_date, gradmaj, instregion, stufttr, previous_2_years, stutotsch, case 
    WHEN stuyear is null THEN 0 
    ELSE CONVERT(datetime, CONCAT((new_month), '/', '01', '/', stuyear)) end as enroll_date
    FROM new_table
)
SELECT TOP 5 *
FROM combined
WHERE previous_2_years <= enroll_date and enroll_date <= grad_date
"
dbGetQuery(con, qry)

In [ ]:
# within those two years, count the number of semesters and number of total credit hours
# also look for the highest transfer indicator (anything above 000001 is a transfer)
qry <- "
with new_table as (
    SELECT gradid, gradfice, grad_date, SUBSTRING(gradmaj, 1, 2) as gradmaj, gradtypi, instregion, stufttr, stutotsch, stusem, stuyear, dateadd(yyyy ,-2, grad_date) as 'previous_2_years',
    CASE 
        WHEN stusem = 2 THEN 4 
        WHEN stusem = 1 THEN 10 
        when stusem = 3 THEN 7 
        ELSE 7 
        END as new_month 
    FROM tr_tx_2021.dbo.grads17_dated cohort
    LEFT JOIN ds_tx_thecb.dbo.institutions inst
    ON cohort.gradfice = inst.instfice 
    LEFT JOIN ds_tx_thecb.dbo.enrollments enroll
    ON cohort.gradid = enroll.studentid 
),
combined as (
    SELECT gradid, gradfice, gradtypi, grad_date, gradmaj, instregion, stufttr, previous_2_years, stutotsch, case 
    WHEN stuyear is null THEN 0 
    ELSE CONVERT(datetime, CONCAT((new_month), '/', '01', '/', stuyear)) end as enroll_date
    FROM new_table
)
SELECT TOP 5 gradid, gradtypi, gradmaj, instregion, COUNT(DISTINCT(enroll_date)) as total_sems, sum(stutotsch) as total_hours,
CASE
    WHEN max(stufttr) is null THEN 0
    WHEN max(stufttr) = 000000 THEN 0 
    WHEN max(stufttr) = 000001 THEN 0
    ELSE 1
    END as transfer_ind
FROM combined
WHERE previous_2_years <= enroll_date and enroll_date <= grad_date
GROUP BY gradid, instregion, gradtypi, gradmaj
"
dbGetQuery(con, qry)

In [ ]:
# those that didn't have enrollment info in this time period are filtered out in agg_filtered
# left join agg_filtered results back to combined, which has everyone's info
qry <- "
with new_table as (
    SELECT gradid, gradfice, grad_date, SUBSTRING(gradmaj, 1, 2) as gradmaj, gradtypi, instregion, stufttr, stutotsch, stusem, stuyear, dateadd(yyyy ,-2, grad_date) as 'previous_2_years',
    CASE 
        WHEN stusem = 2 THEN 4 
        WHEN stusem = 1 THEN 10 
        when stusem = 3 THEN 7 
        ELSE 7 
        END as new_month 
    FROM tr_tx_2021.dbo.grads17_dated cohort
    LEFT JOIN ds_tx_thecb.dbo.institutions inst
    ON cohort.gradfice = inst.instfice 
    LEFT JOIN ds_tx_thecb.dbo.enrollments enroll
    ON cohort.gradid = enroll.studentid 
),
combined as (
    SELECT gradid, gradfice, gradtypi, grad_date, gradmaj, instregion, stufttr, previous_2_years, stutotsch, case 
    WHEN stuyear is null THEN 0 
    ELSE CONVERT(datetime, CONCAT((new_month), '/', '01', '/', stuyear)) end as enroll_date
    FROM new_table
),
agg_filtered as (
    SELECT gradid, gradtypi, gradmaj, instregion, COUNT(DISTINCT(enroll_date)) as total_sems, sum(stutotsch) as total_hours,
    CASE
        WHEN max(stufttr) is null THEN 0
        WHEN max(stufttr) = 000000 THEN 0 
        WHEN max(stufttr) = 000001 THEN 0
    ELSE 1
    END as transfer_ind
    FROM combined
    WHERE previous_2_years <= enroll_date and enroll_date <= grad_date
    GROUP BY gradid, instregion, gradtypi, gradmaj
)
SELECT TOP 5 *
FROM combined c
LEFT JOIN agg_filtered a
ON a.gradid = c.gradid
"
dbGetQuery(con, qry)

In [ ]:
# for those who were joined back in, set calculated variables in agg_filtered to 0 (total_sems, total_hours, transfer_ind)
# also only take one record per person using distinct
qry <- "
with new_table as (
    SELECT gradid, gradfice, grad_date, SUBSTRING(gradmaj, 1, 2) as gradmaj, gradtypi, instregion, stufttr, stutotsch, stusem, stuyear, dateadd(yyyy ,-2, grad_date) as 'previous_2_years',
    CASE 
        WHEN stusem = 2 THEN 4 
        WHEN stusem = 1 THEN 10 
        when stusem = 3 THEN 7 
        ELSE 7 
        END as new_month 
    FROM tr_tx_2021.dbo.grads17_dated cohort
    LEFT JOIN ds_tx_thecb.dbo.institutions inst
    ON cohort.gradfice = inst.instfice 
    LEFT JOIN ds_tx_thecb.dbo.enrollments enroll
    ON cohort.gradid = enroll.studentid 
),
combined as (
    SELECT gradid, gradfice, gradtypi, grad_date, gradmaj, instregion, stufttr, previous_2_years, stutotsch, case 
    WHEN stuyear is null THEN 0 
    ELSE CONVERT(datetime, CONCAT((new_month), '/', '01', '/', stuyear)) end as enroll_date
    FROM new_table
),
agg_filtered as (
    SELECT gradid, gradtypi, gradmaj, instregion, COUNT(DISTINCT(enroll_date)) as total_sems, sum(stutotsch) as total_hours,
    CASE
        WHEN max(stufttr) is null THEN 0
        WHEN max(stufttr) = 000000 THEN 0 
        WHEN max(stufttr) = 000001 THEN 0
        ELSE 1
        END as transfer_ind
    FROM combined
    WHERE previous_2_years <= enroll_date and enroll_date <= grad_date
    GROUP BY gradid, instregion, gradtypi, gradmaj
)
SELECT DISTINCT TOP 5 c.gradid, c.gradtypi, SUBSTRING(c.gradmaj, 1, 2) as gradmaj, c.instregion,
    CASE WHEN a.transfer_ind is null THEN 0 ELSE a.transfer_ind end as transfer_ind,
    CASE WHEN a.total_sems is null THEN 0 ELSE a.total_sems end as total_sems,
    CASE WHEN a.total_hours is null THEN 0 ELSE a.total_hours end as total_hours
FROM combined c
LEFT JOIN agg_filtered a
ON a.gradid = c.gradid
"
dbGetQuery(con, qry)

<a id='final-query'></a>

In [ ]:
# save to cohort_2017 data frame
qry <- "
with new_table as (
    SELECT gradid, gradfice, grad_date, SUBSTRING(gradmaj, 1, 2) as gradmaj, gradtypi, instregion, stufttr, stutotsch, stusem, stuyear, dateadd(yyyy ,-2, grad_date) as 'previous_2_years',
    CASE 
        WHEN stusem = 2 THEN 4 
        WHEN stusem = 1 THEN 10 
        when stusem = 3 THEN 7 
        ELSE 7 
        END as new_month 
    FROM tr_tx_2021.dbo.grads17_dated cohort
    LEFT JOIN ds_tx_thecb.dbo.institutions inst
    ON cohort.gradfice = inst.instfice 
    LEFT JOIN ds_tx_thecb.dbo.enrollments enroll
    ON cohort.gradid = enroll.studentid 
),
combined as (
    SELECT gradid, gradfice, gradtypi, grad_date, gradmaj, instregion, stufttr, previous_2_years, stutotsch, case 
    WHEN stuyear is null then 0 
    ELSE CONVERT(datetime, CONCAT((new_month), '/', '01', '/', stuyear)) end as enroll_date
    FROM new_table
),
agg_filtered as (
    SELECT gradid, gradtypi, gradmaj, instregion, COUNT(DISTINCT(enroll_date)) as total_sems, sum(stutotsch) as total_hours,
    CASE
        WHEN max(stufttr) is null THEN 0
        WHEN max(stufttr) = 000000 THEN 0 
        WHEN max(stufttr) = 000001 THEN 0
    ELSE 1
    END as transfer_ind
    FROM combined
    WHERE previous_2_years <= enroll_date and enroll_date <= grad_date
    GROUP BY gradid, instregion, gradtypi, gradmaj
)
SELECT DISTINCT c.gradid, c.gradtypi, SUBSTRING(c.gradmaj, 1, 2) as gradmaj, c.instregion,
    CASE WHEN a.transfer_ind is null THEN 0 ELSE a.transfer_ind end as transfer_ind,
    CASE WHEN a.total_sems is null THEN 0 ELSE a.total_sems end as total_sems,
    CASE WHEN a.total_hours is null THEN 0 ELSE a.total_hours end as total_hours
FROM combined c
LEFT JOIN agg_filtered a
ON a.gradid = c.gradid
"
cohort_2017 <- dbGetQuery(con, qry)

To confirm we did not double count or ignore any individuals from the original cohort, we can compare the number of individuals in `grads17_dated` with the amount of individuals and rows in `cohort_2017`.

In [ ]:
# number of individuals in original cohort
qry <- "
SELECT COUNT(DISTINCT(gradid)) FROM tr_tx_2021.dbo.grads17_dated
"
dbGetQuery(con, qry)

In [ ]:
# number of rows and individuals in indvidual-based variables
cohort_2017 %>%
    summarize(
        number_rows = n(),
        number_individuals = n_distinct(gradid)
    )

### Information from the Past

Recall that there were 2 more features we wanted to create to help us predict our outcome, `employed_count` and `employed_prop`. If we were to try to predict if each individual in the 2017 cohort would be employed in their fourth quarter after graduation, we could only use information avaialable to us at the time of their graduation. Given that our outcome variable is in the fourth quarter after graduation, we could not analyze employment outcomes from the 2016 cohort, as we would be working with incomplete information (Four quarters after Fall 2016 is Fall 2017 and there are Spring 2017 graduates). 

Therefore, the most recent information we can utilize would be employment outcomes from the 2015 cohort. We purposely include these two features to begin to understand the different labor market dynamics in the regions of Texas for recent bachelor's degree recipients. 

Recall that the employment outcomes for the 2015 calendar year graduates are stored in the table `nb_cohort_wages_link`.

In [ ]:
# see nb_cohort_wages_link table
qry <- "
SELECT TOP 5 *
FROM tr_tx_2021.dbo.nb_cohort_wages_link
"
dbGetQuery(con, qry)

We can add in the insitution-level information, as well as restrict employment to the fourth quarter after graduation with following query.

In [ ]:
# get information on graduating institution
qry <- "
SELECT TOP 5 w.gradid, w.gradtypi, substring(w.gradmaj,1, 2) as gradmaj, i.instregion, w.wage 
FROM tr_tx_2021.dbo.nb_cohort_wages_link w 
LEFT JOIN ds_tx_thecb.dbo.institutions i 
ON w.gradfice = i.instfice 
WHERE dateadd(quarter, 4, grad_date) = job_date
"
dbGetQuery(con, qry)

We can then aggregate this information in a CTE to find the number of individuals from the 2015 cohort that experienced employment in their fourth quarter after graduation by the region of their graduating institution and their degree.

In [ ]:
# aggregate to gradmaj and instregion and count the number of indivdiuals within these combinations
qry <- "
with init as (
    SELECT w.gradid, w.gradtypi, SUBSTRING(w.gradmaj,1, 2) as gradmaj, i.instregion, w.wage 
    FROM tr_tx_2021.dbo.nb_cohort_wages_link w 
    LEFT JOIN ds_tx_thecb.dbo.institutions i 
    ON w.gradfice = i.instfice 
    WHERE dateadd(quarter, 4, grad_date) = job_date
)
SELECT COUNT(DISTINCT(gradid)) as employed_count, gradmaj, instregion 
FROM init 
GROUP BY gradmaj, instregion
"
additional_features <- dbGetQuery(con, qry)

head(additional_features)

In [ ]:
# add in employed_prop
additional_features <- additional_features %>%
    group_by(instregion) %>% 
    mutate(
        employed_prop = round(employed_count/sum(employed_count), digits = 4)
    ) %>%
    ungroup()

head(additional_features)

We can then join this aggregated information from the 2015 cohort back to our 2017 cohort on the `gradmaj` and `instregion` variables.

In [ ]:
# join features
training_set <- cohort_2017 %>% 
    left_join(
        additional_features, 
        by=c("instregion" = "instregion", "gradmaj" = "gradmaj")
    )

training_set %>% head()

In [ ]:
# some graduates have major/institution region combos that didn't have any employment in 2015 so they will be set to 0
training_set <- training_set %>%
    mutate(
        employed_count = ifelse(is.na(employed_count), 0, employed_count),
        employed_prop = ifelse(is.na(employed_prop), 0, employed_prop)
    )

## **Outcome Variable**

With the features stored in `training_set`, we just need to add one last variable to the data frame before we begin running supervised machine learning algorithms: if they ended up as employed in their fourth quarter after graduation.

We have created a de-duplicated, dated, and subset version of the wage records to join to the `grads17_dated` table.

In [ ]:
# link 2017 cohort to wage records in 4th quarter after graduation
# only taking distinct so that we don't double-count anyone
qry <- "
SELECT DISTINCT cohort.gradid, 1 as wage_ind
FROM tr_tx_2021.dbo.grads17_dated cohort
JOIN tr_tx_2021.dbo.wage_record17_dated_dedup w
ON cohort.gradid  = w.ssn
WHERE dateadd(quarter, 4, grad_date) = job_date
"

training_wages <- dbGetQuery(con, qry)

In [ ]:
# see amount of individuals in training_wages
# note that this is not everyone in the cohort, just those that did experience employment
training_wages %>% 
    summarize(
        number_individuals = n_distinct(gradid)
    )

In [ ]:
# find those that didn't show up in wage records
# give them a 0 for wage indicator
training_no_wages <- training_set %>%
    anti_join(training_wages, by = "gradid") %>%
    distinct(gradid)  %>% 
    mutate(
        wage_ind = 0
    )

In [ ]:
# combine two data frames with outcomes
training_outcome <- training_wages %>%
    rbind(training_no_wages)

In [ ]:
# join back to training features by gradid
full_training_set <- training_set %>% 
    inner_join(training_outcome, by = ("gradid" = "gradid")) 

head(full_training_set)

In [ ]:
# make sure we didn't lose anyone in inner_join (aka everyone has an outcome variable)
full_training_set %>%
    summarize(
        number_rows = n(),
        number_individuals = n_distinct(gradid)
    )

We already saved the training set as `nb_training_set` in the `tr_tx_2021` database. The process for creating the testing set follows the same steps, just one year later. You can test your code by recreating `full_training_set` for the 2018 cohort and comparing it to the permanent table `nb_testing_set`.